<a href="https://colab.research.google.com/github/marco-siino/McRock-SemEval-2022-Task4/blob/main/SemEval2022_Task4_SubTask1_Submission2_MSiino_HybridLSTM_v_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main imports and code

In [ ]:
#from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
from tensorflow.keras import layers
from tensorflow import keras

from tensorflow.keras import losses
from tensorflow.keras import preprocessing

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import torch
from collections import Counter
from ast import literal_eval
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from textblob import TextBlob
nltk.download('stopwords')
nltk.download('punkt')
from textblob import TextBlob

import matplotlib.pyplot as plt
import os
import random
import re
import shutil
import string
import tensorflow as tf
import nltk
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from google.colab import files
from io import open
from numpy.random import seed
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Fetch all the files needed.

In [ ]:
urlDontPatronizeMe_PCL_tsv = "https://raw.githubusercontent.com/marco-siino/McRock-SemEval-2022-Task4/refs/heads/main/dontpatronizeme_pcl.tsv"
urlDontPatronizeMe_categories_tsv = "https://raw.githubusercontent.com/marco-siino/McRock-SemEval-2022-Task4/refs/heads/main/dontpatronizeme_categories.tsv"
urlTestSet_csv = "https://raw.githubusercontent.com/marco-siino/McRock-SemEval-2022-Task4/refs/heads/main/task4_test.csv"
urlDevSet_csv = "https://raw.githubusercontent.com/marco-siino/McRock-SemEval-2022-Task4/refs/heads/main/dev_semeval_parids-labels.csv"
urlTrainSet_csv = "https://raw.githubusercontent.com/marco-siino/McRock-SemEval-2022-Task4/refs/heads/main/train_semeval_parids-labels.csv"

tmp = tf.keras.utils.get_file("dontpatronizeme_pcl.tsv", urlDontPatronizeMe_PCL_tsv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("dontpatronizeme_categories.tsv", urlDontPatronizeMe_categories_tsv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("task4_test.csv", urlTestSet_csv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("dev_semeval_parids-labels.csv", urlDevSet_csv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("train_semeval_parids-labels.csv", urlTrainSet_csv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')


253952/241839 [===============================] - 0s 0us/step


# Fetch Don't Patronize Me! data manager module

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


# The official scorer

The script `evaluation.py` can also be obtained by running the cell below:

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py


In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

# Import and load dpm

In [ ]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


# Load paragraph IDs

In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

In [ ]:
print(trids.head())
print(len(trids))
print(len(teids))

   par_id                  label
0    4341  [1, 0, 0, 1, 0, 0, 0]
1    4136  [0, 1, 0, 0, 0, 0, 0]
2   10352  [1, 0, 0, 0, 0, 1, 0]
3    8279  [0, 0, 0, 1, 0, 0, 0]
4    1164  [1, 0, 0, 1, 1, 1, 0]
8375
2094


In [ ]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

# Rebuild training set (Task 1)

In [ ]:
rows = [] # will contain par_id, keyword, country, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  rows.append({
      'par_id':parid,
      'keyword':keyword,
      'country':country,
      'text':text,
      'label':label
  })


In [ ]:
trdf1 = pd.DataFrame(rows)

In [ ]:
len(rows)

8375

In [ ]:
print(trdf1)

     par_id  ... label
0      4341  ...     1
1      4136  ...     1
2     10352  ...     1
3      8279  ...     1
4      1164  ...     1
...     ...  ...   ...
8370   8380  ...     0
8371   8381  ...     0
8372   8382  ...     0
8373   8383  ...     0
8374   8384  ...     0

[8375 rows x 5 columns]


# Rebuild test set (Task 1)

In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  rows.append({
      'par_id':parid,
      'keyword':keyword,
      'country':country,
      'text':text,
      'label':label
  })


In [ ]:
len(rows)

2094

In [ ]:
tedf1 = pd.DataFrame(rows)

# DONT EXECUTE!!! Subsampling negative instances (DS is unbalanced!)

In [ ]:
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

trdf1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*6]])

print(trdf1)

     par_id  ... label
0      4341  ...     1
1      4136  ...     1
2     10352  ...     1
3      8279  ...     1
4      1164  ...     1
...     ...  ...   ...
5553   5253  ...     0
5554   5254  ...     0
5555   5255  ...     0
5556   5256  ...     0
5557   5257  ...     0

[5558 rows x 5 columns]


# Preprocessing functions definitions

In [ ]:
# Do-nothing function.
def do_nothing(input_data):

  return input_data

# Lowercasing function.
def lowercase(input_data):

  return tf.strings.lower(input_data)

# Marks, special characters etc... removing function.
def only_alphanumeric(input_data):

  output_data = tf.strings.regex_replace(input_data, r'[^a-zA-Z0-9\s]', ' ')

  return  output_data

# Stop word removal function.
def stop_word_removal(input_data):

  output_data = input_data

  #print("\n\nInput data è il seguente tensore:")
  #print(output_data)

  #print("Lo converto in stringa e diventa:")
  # Il seguente try per l'adattamento del ts. Nell'except caso della simulazione vera e propria.
  try:
    input_string=output_data[0]

  # # # # # # # Questo è il caso della chiamata a funzione per la simulazione vera e propria.
  except:
    #print("\n\n****CASO DELLA SIMULAZIONE VERA E PROPRIA****\n\n")
    #print("\nQuesto è il contenuto di output data in caso di simulazione")
    #print(output_data)
    input_string=output_data

    try:
      input_string = input_string.numpy()

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      #print("\nestraendo il contenuto del tensore risulta:")
      #print(input_string)
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    if re.search('.*<author_lang="es">.*',input_string):
      outputlist = [word for word in blob if word not in stopwords.words('spanish')]
      #print("tolte le stopword spagnole diventa:")
    else:
      outputlist = [word for word in blob if word not in stopwords.words('english')]
      #print("tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant(output_string)
    #print(output_tensor)

    return output_tensor

   # # # # # # # Questo è il caso dell'adattamento del TS.
  else:

    try:
      input_string = input_string.numpy()[0]
      #print(input_string)

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    if re.search('.*<author_lang="es">.*',input_string):
      outputlist = [word for word in blob if word not in stopwords.words('spanish')]
      #print("tolte le stopword spagnole diventa:")
    else:
      outputlist = [word for word in blob if word not in stopwords.words('english')]
      #print("tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant([[output_string]])
    #print(output_tensor)

    return output_tensor


# Preprocess and adapt function

In [ ]:
max_sample_length=0
def preprocess_and_adapt_ts(preprocessing_function,training_set):
  # Set a very large sequence length to find the longest sample in the training set.
  sequence_length = 1000
  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=sequence_length)

  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  #vectorize_layer.get_vocabulary()

  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
  model.add(vectorize_layer)

  longest_sample_length=1

  for element in training_set:
    authorDocument=element[0]
    label=element[1]

    #print("Sample considered is: ", authorDocument[0].numpy())
    #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
    #print("And has label: ", label[0].numpy())

    # Count the number of zeros from the last non-zero token to the end of the sample.
    # Shortest tokenized sample has less zeros than others.
    out=model(authorDocument)
    token_nr_index=sequence_length-1
    current_sample_zeros_counter=0
    while out.numpy()[0][token_nr_index]==0:
      token_nr_index-=1
      current_sample_zeros_counter+=1

    shortest_padding_length=sequence_length-longest_sample_length
    if current_sample_zeros_counter<shortest_padding_length:
      longest_sample_length=sequence_length-current_sample_zeros_counter

  #print(out.numpy()[0][3229:3400])
  #print(longest_sample_length)

  # After tokenization longest_sample_length covers all the document lenghts in our dataset.
  global max_sample_length
  max_sample_length = longest_sample_length

  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=max_sample_length)

  # Finally adapt the vectorize layer.
  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  return vectorize_layer

# Build a Tensorflow DS for Task1

In [ ]:
train_set1 = False
for i in range(0,len(trdf1)):
  sample = [' '+trdf1['keyword'][i]+' '+trdf1['country'][i]+' '+trdf1['text'][i]]
  label = [trdf1['label'][i]]

  current_set = (
    tf.data.Dataset.from_tensor_slices(
        (
            [tf.cast(sample,tf.string)],
            [tf.cast(label, tf.int32)]
        )
    )
  )
  if train_set1 != False:
    train_set1 = train_set1.concatenate(current_set)
  else:
    train_set1 = current_set
train_set1 = train_set1.shuffle(len(train_set1), reshuffle_each_iteration=False)

dev_set1 = False
for i in range(0,len(tedf1)):
  sample = [' '+tedf1['keyword'][i]+' '+tedf1['country'][i]+' '+tedf1['text'][i]]
  label = [tedf1['label'][i]]

  current_set = (
    tf.data.Dataset.from_tensor_slices(
        (
            [tf.cast(sample,tf.string)],
            [tf.cast(label, tf.int32)]
        )
    )
  )
  if dev_set1 != False:
    dev_set1 = dev_set1.concatenate(current_set)
  else:
    dev_set1 = current_set

for element in train_set1:
    authorDocument=element[0]
    label=element[1]
    print(authorDocument)
    print(label)
    break

print(train_set1)


tf.Tensor([b" immigrant gh the suggestion by frauke petry , leader of the anti-immigrant alternative for germany ( afd ) party , has fuelled an already heated debate about chancellor angela merkel 's decision to open germany 's doors to refugees ."], shape=(1,), dtype=string)
tf.Tensor([0], shape=(1,), dtype=int32)
<ShuffleDataset shapes: ((1,), (1,)), types: (tf.string, tf.int32)>


# Vectorize layer adapting

In [ ]:
vectorize_layer = preprocess_and_adapt_ts(do_nothing,train_set1)

# Models definition for Task 1

In [ ]:
embedding_dim = 50
seed = 841321724
initializer=tf.keras.initializers.GlorotUniform(seed=seed)

## Simple ANN

In [ ]:
model1 = tf.keras.Sequential([
  tf.keras.Input(shape=(), dtype=tf.string),
  vectorize_layer,
  layers.Embedding(len(vectorize_layer.get_vocabulary()) + 1,embedding_dim,embeddings_initializer=initializer),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)
  ])


## CNN

In [ ]:
model1 = tf.keras.Sequential([
                              tf.keras.Input(shape=(), dtype=tf.string),
                              vectorize_layer,
                              layers.Embedding(len(vectorize_layer.get_vocabulary()) + 1,embedding_dim,embeddings_initializer=initializer),
                              #layers.Dropout(0.5),
                              layers.Conv1D(32,12,kernel_initializer=initializer,bias_initializer='zeros',activation='relu'),
                              #layers.Dropout(0.5),
                              layers.MaxPooling1D(4),
                              #layers.AveragePooling1D(8),
                              layers.GlobalAveragePooling1D(),
                              #layers.GlobalMaxPooling1D(),
                              layers.Dense(1)
        ])

## Multi-Channel CNN

In [ ]:
input= tf.keras.Input(shape=(1,), dtype=tf.string)
x=vectorize_layer(input)

y1=layers.Embedding(len(vectorize_layer.get_vocabulary()) + 1, embedding_dim)(x)
y1=layers.Dropout(0.8)(y1)

y2=layers.Embedding(len(vectorize_layer.get_vocabulary()) + 1, embedding_dim)(x)
y2=layers.Dropout(0.8)(y2)

y3=layers.Embedding(len(vectorize_layer.get_vocabulary()) + 1, embedding_dim)(x)
y3=layers.Dropout(0.8)(y3)

y4=layers.Embedding(len(vectorize_layer.get_vocabulary()) + 1, embedding_dim)(x)
y4=layers.Dropout(0.8)(y4)

y1=layers.Conv1D(32,32)(y1)
y1=layers.MaxPooling1D()(y1)
y1=layers.Dropout(0.5)(y1)
y1=layers.Flatten()(y1)

y2=layers.Conv1D(32,16)(y2)
y2=layers.MaxPooling1D()(y2)
y2=layers.Dropout(0.5)(y2)
y2=layers.Flatten()(y2)

y3=layers.Conv1D(32,2)(y3)
y3=layers.MaxPooling1D()(y3)
y3=layers.Dropout(0.5)(y3)
y3=layers.Flatten()(y3)

y4=layers.Conv1D(32,1)(y4)
y4=layers.MaxPooling1D()(y4)
y4=layers.Dropout(0.5)(y4)
y4=layers.Flatten()(y4)

merged = layers.concatenate([y1,y2,y3,y4])

x=layers.Dense(300)(merged)
x=layers.Dropout(0.5)(x)
output=layers.Dense(1)(x)
model1 = tf.keras.Model(inputs=input, outputs=output)

## Hybrid LSTM

In [ ]:
# Hybrid LSTM
model1 = tf.keras.Sequential([
                              tf.keras.Input(shape=(), dtype=tf.string),
                              vectorize_layer,
                              layers.Embedding(len(vectorize_layer.get_vocabulary()) + 1,embedding_dim,embeddings_initializer=initializer),
                              layers.Conv1D(64,2,activation='relu'),
                              layers.Dropout(0.5),
                              layers.MaxPooling1D(4),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                              tf.keras.layers.Dense(64, activation='relu'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(1)
                              ])


## Transformers

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(), dtype=tf.string)
x = vectorize_layer(inputs)
embedding_layer = TokenAndPositionEmbedding(max_sample_length, len(vectorize_layer.get_vocabulary()) + 1, embedding_dim)
x = embedding_layer(x)
transformer_block = TransformerBlock(embedding_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1)(x)

model1 = tf.keras.Model(inputs=inputs, outputs=outputs)

# Model compiling and training for Task 1

In [ ]:
opt = tf.keras.optimizers.Adam()
metric = tf.keras.metrics.BinaryAccuracy()
ls = tf.keras.losses.BinaryCrossentropy(from_logits=True)

model1.compile(loss=ls, optimizer=opt, metrics=metric)
#model1.summary()

np.random.seed(1)
random.seed(2)
tf.random.set_seed(1234)

epochs = 20

"""
for element in train_set1:
    authorDocument=element[0]
    label=element[1]
    print(authorDocument)
    print(label)
    break
"""
train_set1_batched = train_set1.batch(100)
history = model1.fit(
          train_set1_batched,
          validation_data=dev_set1,
          epochs=epochs,
          shuffle=True,
          # Comment the following line to do not save and download the model.
          #callbacks=[callbacks]
          )

Epoch 1/20
84/84 [==============================] - 137s 1s/step - loss: 0.3572 - binary_accuracy: 0.9052 - val_loss: 0.3130 - val_binary_accuracy: 0.9050
Epoch 2/20
84/84 [==============================] - 123s 1s/step - loss: 0.2828 - binary_accuracy: 0.9057 - val_loss: 0.2837 - val_binary_accuracy: 0.9050
Epoch 3/20
84/84 [==============================] - 123s 1s/step - loss: 0.1497 - binary_accuracy: 0.9465 - val_loss: 0.3092 - val_binary_accuracy: 0.9021
Epoch 4/20
84/84 [==============================] - 124s 1s/step - loss: 0.0589 - binary_accuracy: 0.9813 - val_loss: 0.4795 - val_binary_accuracy: 0.9007
Epoch 5/20
84/84 [==============================] - 123s 1s/step - loss: 0.0252 - binary_accuracy: 0.9926 - val_loss: 0.5376 - val_binary_accuracy: 0.8902
Epoch 6/20
84/84 [==============================] - 123s 1s/step - loss: 0.0137 - binary_accuracy: 0.9963 - val_loss: 0.6763 - val_binary_accuracy: 0.8649
Epoch 7/20
84/84 [==============================] - 124s 1s/step - los

# Print predictions on train_set1

In [ ]:
for element in train_set1:
    authorDocument=element[0]
    label=element[1]
    print(authorDocument)
    print(label)
    print(model1.predict(authorDocument))


tf.Tensor([b' poor-families pk "quetta : pakistan tehreek-e-insaf ( pti ) balochistan president yar muhammad rind has said that pti will be victorious on july 25 . "" after the victory , education and health sectors will be given proper attention as we shall hand over pens to the children of poor families , "" said the pti provincial president while addressing a corner meeting in naseerabad on monday ."'], shape=(1,), dtype=string)
tf.Tensor([1], shape=(1,), dtype=int32)
[[9.221133]]
tf.Tensor([b' poor-families za zuma also took a swipe at the institutions of higher learning for depriving students from poor families a chance to be educated .'], shape=(1,), dtype=string)
tf.Tensor([0], shape=(1,), dtype=int32)
[[-3.8730648]]
tf.Tensor([b' disabled gh he said in the case of the disabled children , when their parents were traced and interrogated , their responses often showed that they deliberately abandoned their children .'], shape=(1,), dtype=string)
tf.Tensor([0], shape=(1,), dtype=in

KeyboardInterrupt: ignored

# Make folders for gold and predicted labels

In [ ]:
# first, we need to create the res/ and ref/ folders, which the evaluator expects
!mkdir ref res

# Make predictions on dev_set1 and write to task1.txt

In [ ]:
preds_task1 = model1.predict(dev_set1)

print(preds_task1)

predictions = []
for current_pred in preds_task1:
        if current_pred[0]>0.0:
          predictions.append([1])
        else:
          predictions.append([0])
predictions = np.array(predictions)
print(predictions)
for pred in preds_task1:
  print(pred)
  break

labels2file(predictions, os.path.join('res/', 'task1.txt'))


[[  2.038012 ]
 [  5.1470246]
 [ -4.981225 ]
 ...
 [ -9.668268 ]
 [-11.578417 ]
 [-11.30858  ]]
[[1]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
[2.038012]


# Evaluate predictions

After generating two prediction files (`task1.txt` and `task2.txt`), we can just call the scorer (`evaluation.py`), which will generate a `scores.txt` file.

# Get gold labels

We will use the gold labels of the training set and compare our random predictions to them.

In [ ]:
# for subtask 1 (we convert our list of labels into a list of lists to make
# it compatible with the labels2file function)
labels2file(tedf1.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task1.txt'))
# and for subtask 2
#labels2file(tedf2.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task2.txt'))
#labels2file(dpm.train_task2_df.label.tolist(), os.path.join('ref/', 'task2.txt'))

In [ ]:
# Now, we can just call the official scorer, which takes an input_directory and an output_directory
# as arguments. In this example, both will be the root directory of this notebook.
!python3 evaluation.py . .

In [ ]:
# CONFIGURAZIONE ATTUALE DA BATTERE!
# The scorer generated a results file called "scores.txt".
# We can now see the performance of a random baseline on the training set.
!cat scores.txt

task1_precision:0.3146853146853147
task1_recall:0.22613065326633167
task1_f1:0.2631578947368421


# Build TF dataset from Test Set

In [ ]:
test_df = pd.read_csv('task4_test.csv',sep='\t', header=None)

print(test_df.head())

test_set = False
for i in range(0,len(test_df)):
  sample = [test_df[2][i]+' '+test_df[3][i]+' '+test_df[4][i]]
  #label = [training_set1['label'][i]]

  current_set = (
    tf.data.Dataset.from_tensor_slices(
        (
            [tf.cast(sample,tf.string)],
            #[tf.cast(label, tf.int32)]
        )
    )
  )
  if test_set != False:
    test_set = test_set.concatenate(current_set)
  else:
    test_set = current_set

for element in test_set:
    authorDocument=element[0]
    #label=element[1]
    print(authorDocument)
    #print(label)
    break

print(test_set)


     0           1  ...   3                                                  4
0  t_0   @@7258997  ...  us  In the meantime , conservatives are working to...
1  t_1  @@16397324  ...  pk  In most poor households with no education chil...
2  t_2  @@16257812  ...  ca  The real question is not whether immigration i...
3  t_3   @@3509652  ...  gb  In total , the country 's immigrant population...
4  t_4    @@477506  ...  ca  Members of the church , which is part of Ken C...

[5 rows x 5 columns]
tf.Tensor([b"vulnerable us In the meantime , conservatives are working to weaken Clinton and drive down her numbers in early voting states , where she is increasingly vulnerable . They are , in effect , doing Sanders 's dirty work for him while he avoids scrutiny ."], shape=(1,), dtype=string)
<ConcatenateDataset shapes: ((1,),), types: (tf.string,)>


# Build the full training set(train+dev) and adapt vectorize layer.




In [ ]:
full_train_set1 = train_set1.concatenate(dev_set1)
vectorize_layer = preprocess_and_adapt_ts(do_nothing,full_train_set1)

# Train the model1 on the full training set.

In [ ]:
model1.compile(loss=ls, optimizer=opt, metrics=metric)
#model1.summary()

np.random.seed(1)
random.seed(2)
tf.random.set_seed(1234)

epochs = 10

"""
for element in train_set1:
    authorDocument=element[0]
    label=element[1]
    print(authorDocument)
    print(label)
    break
"""
full_train_set1_batched = full_train_set1.batch(100)
history = model1.fit(
          full_train_set1_batched,
          validation_data=dev_set1,
          epochs=epochs,
          shuffle=True,
          # Comment the following line to do not save and download the model.
          #callbacks=[callbacks]
          )

Epoch 1/10
105/105 [==============================] - 132s 1s/step - loss: 0.2570 - binary_accuracy: 0.9491 - val_loss: 0.7138 - val_binary_accuracy: 0.9050
Epoch 2/10
105/105 [==============================] - 130s 1s/step - loss: 0.1950 - binary_accuracy: 0.9053 - val_loss: 0.3667 - val_binary_accuracy: 0.9050
Epoch 3/10
105/105 [==============================] - 130s 1s/step - loss: 0.1169 - binary_accuracy: 0.9287 - val_loss: 0.1239 - val_binary_accuracy: 0.9398
Epoch 4/10
105/105 [==============================] - 130s 1s/step - loss: 0.0576 - binary_accuracy: 0.9725 - val_loss: 0.0610 - val_binary_accuracy: 0.9833
Epoch 5/10
105/105 [==============================] - 130s 1s/step - loss: 0.0346 - binary_accuracy: 0.9893 - val_loss: 0.0411 - val_binary_accuracy: 0.9900
Epoch 6/10
105/105 [==============================] - 131s 1s/step - loss: 0.0195 - binary_accuracy: 0.9947 - val_loss: 0.0275 - val_binary_accuracy: 0.9933
Epoch 7/10
105/105 [==============================] - 130s

# Make predictions on test_set and write to task1.txt

In [ ]:
preds_task1 = model1.predict(test_set)

print(preds_task1)

predictions = []
for current_pred in preds_task1:
        if current_pred[0]>0.0:
          predictions.append([1])
        else:
          predictions.append([0])
predictions = np.array(predictions)
print(predictions)
for pred in preds_task1:
  print(pred)
  break

labels2file(predictions, os.path.join('res/', 'task1.txt'))


[[-9.290273 ]
 [-4.908674 ]
 [-7.2991085]
 ...
 [-3.0083985]
 [-8.921698 ]
 [-9.205551 ]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
[-9.290273]


# Prepare submission

In [ ]:
!cat res/task1.txt

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0


In [ ]:
!zip submission.zip res/task1.txt #res/task2.txt